In [1]:
# complete example, see https://github.com/tensorflow/models/blob/v1.13.0/research/object_detection/object_detection_tutorial.ipynb
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [2]:
import sys

tf_extern_root = '/opt/tf-object-detection/TensorFlow/models/research/'
sys.path.append(tf_extern_root)
sys.path.append(tf_extern_root + 'slim')
sys.path.append(tf_extern_root + 'object_detection')

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [3]:

# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [4]:
#MODEL_NAME = 'scripts/inference_graph'
model_prefix = '10251753'
MODEL_NAME = 'scripts/trained/' + model_prefix + '/inference_graph'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [7]:
PATH_TO_LABELS = '/share/Competition2/corn_label_map_1class.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [8]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [9]:
import fnmatch
import os
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = '/share/Competition2/images/'

TEST_IMAGE_PATHS = []

'''
for file in os.listdir(PATH_TO_TEST_IMAGES_DIR):
    if fnmatch.fnmatch(file, 'file3[3-4]frame*.jpg'):
        #print(file)
        TEST_IMAGE_PATHS.append(os.path.join(PATH_TO_TEST_IMAGES_DIR,file))
'''
for file in os.listdir(PATH_TO_TEST_IMAGES_DIR):
    if fnmatch.fnmatch(file, '*.jpg'):
        TEST_IMAGE_PATHS.append(os.path.join(PATH_TO_TEST_IMAGES_DIR,file))      

#print(TEST_IMAGE_PATHS) #TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'file{}frame300.jpg'.format(i)) for i in range(30, 34) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [10]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
      #print(output_dict)
  return output_dict

In [ ]:
%matplotlib inline
import os
prediction_dir='data/prediction_' + model_prefix
if not os.path.exists(prediction_dir):
    os.mkdir(prediction_dir)
ii = 0
for image_path in TEST_IMAGE_PATHS:
  print("image:", image_path)
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  if ii < 10:
    plt.imshow(image_np)
  plt.imsave(prediction_dir + "/pred_" + image_path.split("/")[-1], image_np)
  ii = ii+ 1
print("all done!")

image: /share/Competition2/images/file92frame400.jpg
image: /share/Competition2/images/file80frame300.jpg
image: /share/Competition2/images/file89frame200.jpg
image: /share/Competition2/images/file91frame200.jpg
image: /share/Competition2/images/file89frame100.jpg
image: /share/Competition2/images/file92frame300.jpg
image: /share/Competition2/images/file16frame500.jpg
image: /share/Competition2/images/file14frame400.jpg
image: /share/Competition2/images/file7frame300.jpg
image: /share/Competition2/images/file72frame200.jpg
image: /share/Competition2/images/file73frame200.jpg
image: /share/Competition2/images/file70frame200.jpg
image: /share/Competition2/images/file15frame400.jpg
image: /share/Competition2/images/file76frame100.jpg
image: /share/Competition2/images/file5frame400.jpg
image: /share/Competition2/images/file12frame200.jpg
image: /share/Competition2/images/file94frame200.jpg
image: /share/Competition2/images/file21frame300.jpg
